In [1]:
import datetime
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
import tensorflow as tf
import datetime

In [4]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [5]:
# Load and preprocess data
data = pd.read_csv('MERGED_BTC-15m-2015-07-20-to-2024-06-27.csv', parse_dates=True, index_col='datetime')

# Calculate the index for splitting into 75% training and 25% testing
# split_index is the index position of the end of the first 75% of the data
split_index = int(len(data) * 0.75)
# print(split_index)

# Split the data into training and testing sets
# grabs all the data up to the index from split index (first 75%)
data = data.iloc[:split_index]
data = data.dropna()
prices = data['close'].copy()

In [6]:
# Create the dataset with windowing method
def create_dataset(prices, window_size):
    data = []
    for i in range(len(prices) - window_size):
        X = prices[i:i + window_size]
        y = prices[i + window_size]
        data.append(list(X) + [y])
    columns = [f't-{i}' for i in range(window_size, 0, -1)] + ['t']
    return pd.DataFrame(data, columns=columns)

window_size = 40
dataset = create_dataset(prices, window_size)
# print(dataset)
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]
# print(X)
# print(y)

/var/folders/j3/f2pn80cx63z808kw83_jxw3w0000gn/T/ipykernel_40851/94713225.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = prices[i + window_size]


In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Normalize features and target using the training data
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)
y_scaler = MinMaxScaler()
y_scaler.fit(y_train)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

# sanity check
# print(window_size == X_train.shape[1])

In [8]:
# Define the model
model = Sequential()
model.add(Dense(10, input_dim=window_size, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Compile the model with a lower learning rate
model.compile(optimizer=SGD(learning_rate=0.001), loss='mean_squared_error', metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
history = model.fit(X_train_scaled, y_train_scaled, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test_scaled), verbose=1, callbacks=[tensorboard_callback])

Epoch 1/10
5499/5499 ━━━━━━━━━━━━━━━━━━━━ 3s 430us/step - loss: 0.0684 - mean_absolute_percentage_error: 3299.8037 - val_loss: 0.0551 - val_mean_absolute_percentage_error: 2310.5474
Epoch 2/10
5499/5499 ━━━━━━━━━━━━━━━━━━━━ 2s 405us/step - loss: 0.0528 - mean_absolute_percentage_error: 3058.1353 - val_loss: 0.0413 - val_mean_absolute_percentage_error: 2003.7570
Epoch 3/10
5499/5499 ━━━━━━━━━━━━━━━━━━━━ 3s 470us/step - loss: 0.0319 - mean_absolute_percentage_error: 2567.0957 - val_loss: 0.0025 - val_mean_absolute_percentage_error: 622.9353
Epoch 4/10
5499/5499 ━━━━━━━━━━━━━━━━━━━━ 3s 460us/step - loss: 0.0010 - mean_absolute_percentage_error: 450.5447 - val_loss: 1.1337e-04 - val_mean_absolute_percentage_error: 125.0505
Epoch 5/10
5499/5499 ━━━━━━━━━━━━━━━━━━━━ 3s 521us/step - loss: 9.9504e-05 - mean_absolute_percentage_error: 297.9168 - val_loss: 7.0149e-05 - val_mean_absolute_percentage_error: 92.4844
Epoch 6/10
5499/5499 ━━━━━━━━━━━━━━━━━━━━ 3s 456us/step - loss: 6.4833e-05 - mean_ab

## Using TensorBoard with Keras Model.fit()

When training with Keras's [Model.fit()](https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#fit), adding the `tf.keras.callbacks.TensorBoard` callback ensures that logs are created and stored. Additionally, enable histogram computation every epoch with `histogram_freq=1` (this is off by default)

Place the logs in a timestamped subdirectory to allow easy selection of different training runs.

Start TensorBoard through the command line or within a notebook experience. The two interfaces are generally the same. In notebooks, use the `%tensorboard` line magic. On the command line, run the same command without "%".

In [10]:
%tensorboard --logdir logs/fit

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_model_fit.png?raw=1"/> -->

A brief overview of the visualizations created in this example and the dashboards (tabs in top navigation bar) where they can be found:

* **Scalars** show how the loss and metrics change with every epoch. You can use them to also track training speed, learning rate, and other scalar values. Scalars can be found in the **Time Series** or **Scalars** dashboards.
* **Graphs** help you visualize your model. In this case, the Keras graph of layers is shown which can help you ensure it is built correctly. Graphs can be found in the **Graphs** dashboard.
* **Histograms** and **Distributions** show the distribution of a Tensor over time. This can be useful to visualize weights and biases and verify that they are changing in an expected way. Histograms can be found in the **Time Series** or **Histograms** dashboards. Distributions can be found in the **Distributions** dashboard.

Additional TensorBoard dashboards are automatically enabled when you log other types of data. For example, the Keras TensorBoard callback lets you log images and embeddings as well. You can see what other dashboards are available in TensorBoard by clicking on the "inactive" dropdown towards the top right.